# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel. It makes use of the collection of skills that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner).

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.12.207.1-preview"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.CoreSkills;
using Microsoft.SemanticKernel.KernelExtensions;
using Microsoft.SemanticKernel.Orchestration;

IKernel kernel = KernelBuilder.Create();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
if (useAzureOpenAI)
    kernel.Config.AddAzureTextCompletionService("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletionService("davinci", model, apiKey, orgId);

### Setting Up the Planner
The planner is located in the Semantic Kernel's CoreSkills and requires Orchestration

In [ ]:
// Load native skill into the kernel registry, sharing its functions with prompt templates
var planner = kernel.ImportSkill(new PlannerSkill(kernel));

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk.

In [ ]:
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "skills");
kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "SummarizeSkill");
kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "WriterSkill");

Define your ASK. What do you want the Kernel to do?

In [ ]:
var ask = "Tomorrow is Valentine's day. I need to come up with a few date ideas and e-mail them to my significant other.";
var originalPlan = await kernel.RunAsync(ask, planner["CreatePlan"]);

Console.WriteLine("Original plan:\n");
Console.WriteLine(originalPlan.Variables.ToPlan().PlanString);

Original plan:

<goal>
Tomorrow is Valentine's day. I need to come up with a few date ideas and e-mail them to my significant other.
</goal>
<plan>
  <function.WriterSkill.Brainstorm input="Valentine's Day Date Ideas" setContextVariable="DATE_IDEAS"/>
  <function._GLOBAL_FUNCTIONS_.BucketOutputs input="$DATE_IDEAS" bucketCount="3" bucketLabelPrefix="DATE_IDEA"/>
  <function.WriterSkill.EmailTo to="My Significant Other" input="$DATE_IDEA_1;$DATE_IDEA_2;$DATE_IDEA_3" sender="Me" />
</plan>


As you can see in the above plan, the Planner has taken the user's ask and converted it into an XML-based plan detailing how the AI would go about solving this task.

It makes use of the skills that the Kernel has available to it and determines which functions to call in order to fullfill the user's ask.

The output of each step of the plan gets set as `setContextVariable` which makes it available as `input` to the next skill.

Let's also define an inline skill and have it be available to the Planner.
Be sure to give it a function name and skill name.

In [ ]:
string skPrompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
""";
var shakespeareFunction = kernel.CreateSemanticFunction(skPrompt, "shakespeare", "ShakespeareSkill", maxTokens: 2000, temperature: 0.2, topP: 0.5);

Let's update our ask using this new skill.

In [ ]:
var ask = @"Tomorrow is Valentine's day. I need to come up with a few date ideas.
 She likes Shakespeare so write using his style. E-mail these ideas to my significant other";

In [ ]:
var newPlan = await kernel.RunAsync(ask, planner["CreatePlan"]);

In [ ]:
Console.WriteLine("Updated plan:\n");
Console.WriteLine(newPlan.Variables.ToPlan().PlanString);

Updated plan:

<goal>
Tomorrow is Valentine's day. I need to come up with a few date ideas.
 She likes Shakespeare so write using his style. E-mail these ideas to my significant other
</goal>
<plan>
  <function.WriterSkill.Brainstorm input="Valentine's Day Date Ideas" setContextVariable="IDEAS"/>
  <function.ShakespeareSkill.shakespeare input="$IDEAS" setContextVariable="SHAKESPEARE_IDEAS"/>
  <function.WriterSkill.EmailTo to="My Significant Other" input="$SHAKESPEARE_IDEAS" sender="Me" appendToResult="RESULT__EMAIL"/>
</plan>


### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a skill called `ExecutePlan`.

In [ ]:
var executionResults = newPlan;

In [ ]:
int step = 1;
int maxSteps = 10;
while (!executionResults.Variables.ToPlan().IsComplete && step < maxSteps)
{
    var results = await kernel.RunAsync(executionResults.Variables, planner["ExecutePlan"]);
    if (results.Variables.ToPlan().IsSuccessful)
    {
        Console.WriteLine($"Step {step} - Execution results:\n");
        Console.WriteLine(results.Variables.ToPlan().PlanString);

        if (results.Variables.ToPlan().IsComplete)
        {
            Console.WriteLine($"Step {step} - COMPLETE!");
            Console.WriteLine(results.Variables.ToPlan().Result);
            break;
        }
    }
    else
    {
        Console.WriteLine($"Step {step} - Execution failed:");
        Console.WriteLine(results.Variables.ToPlan().Result);
        break;
    }
    
    executionResults = results;
    step++;
    Console.WriteLine("");
}

Step 1 - Execution results:

<goal>
Tomorrow is Valentine's day. I need to come up with a few date ideas.
 She likes Shakespeare so write using his style. E-mail these ideas to my significant other
</goal><plan>
  <function.ShakespeareSkill.shakespeare input="$IDEAS" setContextVariable="SHAKESPEARE_IDEAS" />
  <function.WriterSkill.EmailTo to="My Significant Other" input="$SHAKESPEARE_IDEAS" sender="Me" appendToResult="RESULT__EMAIL" />
</plan>

Step 2 - Execution results:

<goal>
Tomorrow is Valentine's day. I need to come up with a few date ideas.
 She likes Shakespeare so write using his style. E-mail these ideas to my significant other
</goal><plan>
  <function.WriterSkill.EmailTo to="My Significant Other" input="$SHAKESPEARE_IDEAS" sender="Me" appendToResult="RESULT__EMAIL" />
</plan>

Step 3 - Execution results:

<goal>
Tomorrow is Valentine's day. I need to come up with a few date ideas.
 She likes Shakespeare so write using his style. E-mail these ideas to my significant other
